#### Users will need to change the input fields in **User Inputs** section and run the simulation at **result** section.

Go to <a href='#user_input'>user inputs</a> / Go to <a href='#result'>results</a>

In [14]:
import csv
from math import floor
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import copy
import sys
import re
import json
import numpy as np
from datetime import datetime
import xlwt 
from xlwt import Workbook 
import matplotlib.dates as mdates


ModuleNotFoundError: No module named 'matplotlib'

#### Utilities

In [3]:
def csv2list(file):
    with open(file) as f:
        floatlist = [list(map(float,l))[0] for l in csv.reader(f, delimiter=',')]
    return floatlist

def readInputs(file):
    with open(file) as f:
        inputs = json.load(f)
    battCap = float(inputs["batt_kwh"])
    battPower = float(inputs["batt_kw"])
    genPower = float(inputs["generator_kw"])
    fuelavail = float(inputs["fuelavail"])
    pvsize = float(inputs["pv_kw"])
    windsize = float(inputs["wind_kw"])

    return battCap, battPower, genPower, fuelavail, pvsize

def readInputsFromREoptResultsFile(file):
    with open(file) as f:
        inputs = json.load(f)
    battCap = float(inputs["outputs"]["Storage"]["size_kwh"])
    battPower = float(inputs["outputs"]["Storage"]["size_kw"])
    genPower = float(inputs["outputs"]["Generator"]["size_kw"])
    fuelAvail = float(inputs["inputs"]["Generator"]["fuel_avail_gal"])
    pvProd = inputs["outputs"]["PV"]["year_one_power_production_series_kw"]
    windProd = inputs["outputs"]["Wind"]["year_one_power_production_series_kw"]
    load = inputs["inputs"]["LoadProfile"]["loads_kw"]
    soc = inputs["outputs"]["Storage"]["year_one_soc_series_pct"]

    return battCap, battPower, genPower, fuelAvail, pvProd, windProd, load, soc


#### Generator

In [1]:
class generator():
    def __init__(self, diesel_kw, fuel_available, diesel_min_turndown=0.3):
        self.kw = diesel_kw
        self.fuel_available = fuel_available if self.kw > 0 else 0
        m, b = self.default_fuel_burn_rate(diesel_kw)
        self.m = m
        self.b = b * self.kw
        self.min_turndown = diesel_min_turndown
        self.genmin = self.min_turndown * self.kw
        
    def default_fuel_burn_rate(self, diesel_kw):
        if self.kw <= 40:
            m = 0.068
            b = 0.0125
        elif self.kw <= 80:
            m = 0.066
            b = 0.0142
        elif self.kw <= 150:
            m = 0.0644
            b = 0.0095
        elif self.kw <= 250:
            m = 0.0648
            b = 0.0067
        elif self.kw <= 750:
            m = 0.0656
            b = 0.0048
        elif self.kw <= 1500:
            m = 0.0657
            b = 0.0043
        else:
            m = 0.0657
            b = 0.004
        return m, b

    def gen_avail(self, n_steps_per_hour):  # kW
        if self.fuel_available - self.b > 0:
            return min((self.fuel_available * n_steps_per_hour - self.b) / self.m, self.kw)
        else:
            return 0

    def fuel_consume(self, gen_output, n_steps_per_hour):  # kW
        if self.gen_avail(n_steps_per_hour) >= self.genmin and gen_output > 0:
            gen_output = max(self.genmin, min(gen_output, self.gen_avail(n_steps_per_hour)))
            fuel_consume = (self.b + self.m * gen_output) / n_steps_per_hour 
            self.fuel_available -= min(self.fuel_available, fuel_consume)
        else:
            gen_output = 0
        return gen_output


#### Battery

In [2]:
class battery():
    def __init__(self, batt_kwh, batt_kw, batt_roundtrip_efficiency=0.89, soc=1.0):
        self.kw = batt_kw
        self.size = batt_kwh if self.kw > 0 else 0
        self.soc = soc
        self.roundtrip_efficiency = batt_roundtrip_efficiency

    def batt_avail(self, n_steps_per_hour):  # kW
        return min(self.size * self.soc * n_steps_per_hour, self.kw)

    def batt_discharge(self, discharge, n_steps_per_hour):  # kW
        discharge = min(self.batt_avail(n_steps_per_hour), discharge)
        self.soc -= 0 if self.size == 0 else min(discharge / self.size / n_steps_per_hour, self.soc)
        return discharge

    def batt_charge(self, charge, n_steps_per_hour):  # kw
        room = (1 - self.soc)   # if there's room in the battery
        charge = min(room * n_steps_per_hour * self.size / self.roundtrip_efficiency, charge, self.kw / self.roundtrip_efficiency)
        chargesoc = 0 if self.size == 0 else charge * self.roundtrip_efficiency / self.size / n_steps_per_hour
        self.soc += chargesoc
        return charge

#### Dispatch Strategy - Load Following

In [ ]:
def loadFollowing(critical_load, pv, wind, generator, battery, n_steps_per_hour):
    """
    Dispatch strategy for one time step
    """

    # distributed generation minus load is the burden on battery
    unmatch = (critical_load - pv - wind)  # kw
    discharge = 0
    gen_output = 0
    charge = 0
    
    if unmatch < 0:    # pv + wind> critical_load
        # excess PV power to charge battery
        charge = battery.batt_charge(-unmatch, n_steps_per_hour)
        unmatch = 0

    elif generator.genmin <= generator.gen_avail(n_steps_per_hour) and 0 < generator.kw:
        gen_output = generator.fuel_consume(unmatch, n_steps_per_hour)
        # charge battery with excess energy if unmatch < genmin
        charge = battery.batt_charge(max(gen_output - unmatch, 0), n_steps_per_hour)  # prevent negative charge
        discharge = battery.batt_discharge(max(unmatch - gen_output, 0), n_steps_per_hour)  # prevent negative discharge
        unmatch -= (gen_output + discharge - charge)

    elif unmatch <= battery.batt_avail(n_steps_per_hour):   # battery can carry balance
        discharge = battery.batt_discharge(unmatch, n_steps_per_hour)
        unmatch = 0
    
    stat = (gen_output, discharge, charge)
    
    return unmatch, stat, generator, battery

#### Simulator

In [ ]:
def simulator(strategy, generator, battery, critical_loads_kw, pv_kw_ac_hourly, wind_kw_ac_hourly, init_soc):
    n_timesteps = len(critical_loads_kw)
    n_steps_per_hour = n_timesteps / 8760  # type: int

    r = [0] * n_timesteps

    for time_step in range(n_timesteps):
        gen = copy.deepcopy(generator)
        batt = copy.deepcopy(battery)
        # outer loop: do simulation starting at each time step
        batt.soc = init_soc[time_step]   # reset battery for each simulation
                
        for i in range(n_timesteps):    # the i-th time step of simulation
            # inner loop: step through all possible surviving time steps
            # break inner loop if can not survive
            t = (time_step + i) % n_timesteps

            unmatch, stat, gen, batt = strategy(
                        critical_loads_kw[t], pv_kw_ac_hourly[t], wind_kw_ac_hourly[t], gen, batt, n_steps_per_hour)

            if unmatch > 0:  # cannot survive
                r[time_step] = i
                break
            elif (i == (n_timesteps-1)) and (unmatch <=0): # can survive a full year
                r[time_step] = i
                break


    r_min = min(r)
    r_max = max(r)
    r_avg = round((float(sum(r)) / float(len(r))), 2)

    x_vals = range(1, int(floor(r_max)+1))
    y_vals = list()

    for hrs in x_vals:
        y_vals.append(round(float(sum([1 if h >= hrs else 0 for h in r])) / float(n_timesteps), 4))

    return {"resilience_by_timestep": r,
            "resilience_min_timestep": r_min,
            "resilience_max_timestep": r_max,
            "resilience_avg_timestep": r_avg,
            "outage_durations": x_vals,
            "probs_of_surviving": y_vals,
            }

#### Run simulation, plot resilience curve, and write results to excel

In [ ]:
def main(project, tag, json_input_path, battCap, battPower, genPower, fuelAvail, pvSize, windSize, output_path):
    if project not in [None, "", " "]:
        project = project + "-"
        
    if tag not in [None, "", " "]:
        tag = "_" + tag
    
    if output_path in [None, "", " "]:
        output_path = "./"
        
    if read_from_json:
        # battCap, battPower, genPower, fuelavail, pvsize = readInputs(json_input_path)
        battCap, battPower, genPower, fuelAvail, pvProd, windProd, load, soc = readInputsFromREoptResultsFile(json_input_path)
    
    # load = csv2list(load_path)
    n_timesteps = len(load)
    n_steps_per_hour = n_timesteps / 8760
    empty_array = [0] * n_timesteps
        
    # soc = csv2list(soc_path) if soc_path else empty_array
    # pv_factor = csv2list(pv_pf_path) if pv_pf_path else empty_array
    # wind_factor = csv2list(wind_pf_path) if wind_pf_path else empty_array
        
    pv_kw_ac_hourly = pvProd#[f*pvSize for f in pv_factor]
    wind_kw_ac_hourly = windProd#[f*windSize for f in wind_factor]
    
    inputsRE = {
        'batt_kwh': battCap,
        'batt_kw': battPower,
        'pv_kw_ac_hourly': pv_kw_ac_hourly,
        'wind_kw_ac_hourly': wind_kw_ac_hourly,
        'critical_loads_kw': load,
        'init_soc': soc,
        'diesel_kw': genPower,
        'fuel_available': fuelAvail
    }

    inputs = copy.deepcopy(inputsRE)
    inputs['pv_kw_ac_hourly'] = empty_array
    inputs['wind_kw_ac_hourly'] = empty_array
    inputs['init_soc'] = empty_array
    

    REfactor = sum(pv_kw_ac_hourly)/sum(load)
    print("Renewable Energy Factor:{}".format(round(REfactor,3)))

    GEN = generator(**{k: inputs[k] for k in generator.__init__.__code__.co_varnames if k in inputs})
    BATT = battery(**{k: inputs[k] for k in battery.__init__.__code__.co_varnames if k in inputs})

    resp = simulator(loadFollowing, GEN, BATT, **{k: inputs[k] for k in simulator.__code__.co_varnames if k in inputs})
    respRE = simulator(loadFollowing, GEN, BATT, **{k: inputsRE[k] for k in simulator.__code__.co_varnames if k in inputsRE})

    r = resp["resilience_by_timestep"]
    rmax = resp["resilience_max_timestep"]
    y_vals = resp["probs_of_surviving"]
    

    rRE = respRE["resilience_by_timestep"]
    rmaxRE = respRE["resilience_max_timestep"]
    y_valsRE = respRE["probs_of_surviving"]

    def writeResult(resp, file_name, path):
        wb = Workbook() 
        
        sheet1 = wb.add_sheet('Result') 
        sheet1.write(0, 1, 'Timesteps') 
        sheet1.write(1, 0, 'max resilience')
        sheet1.write(1, 1, resp["resilience_max_timestep"])
        sheet1.write(2, 0, 'min resilience')
        sheet1.write(2, 1, resp["resilience_min_timestep"]) 
        sheet1.write(3, 0, 'avg resilience')
        sheet1.write(3, 1, resp["resilience_avg_timestep"])

        sheet2 = wb.add_sheet('resilience_by_timestep') 
        for i, v in enumerate(resp["resilience_by_timestep"]):
            sheet2.write(i, 0, v) 

        sheet3 = wb.add_sheet('probs_of_surviving') 
        for i, v in enumerate(resp["probs_of_surviving"]):
            sheet3.write(i, 0, v)

        wb.save(path + '{}.xls'.format(file_name))
    
    writeResult(resp, "Diesel Only{}".format(tag), output_path)
    writeResult(respRE, "Diesel + RE{}".format(tag), output_path)

    m = int(max(rmax, rmaxRE))
    y_vals = y_vals + [0] * int(m - rmax + 1)
    y_valsRE = y_valsRE + [0] * int(m - rmaxRE + 1)

    hr = [i*3600/n_steps_per_hour for i in range(n_timesteps)]
    x1 = [datetime.utcfromtimestamp(h).strftime("%Y-%m-%dT%H") for h in hr]
    x1 = np.array(x1, dtype='datetime64')
    x2 = [float(i)/float(n_steps_per_hour) for i in range(m+1)]
    
    months = mdates.MonthLocator()  # every month
    monthFmt = mdates.DateFormatter('%b')
    
    r = [float(q)/float(n_steps_per_hour) for q in r] if n_steps_per_hour != 1 else r
    rRE = [float(q)/float(n_steps_per_hour) for q in rRE] if n_steps_per_hour != 1 else rRE
    
    dpi = 200
    fig, ax1 = plt.subplots(figsize=(2200/dpi, 900/dpi), dpi=dpi)
    ax1.fill_between(x1, 0, r, alpha=0.3)
    ax1.fill_between(x1, r, rRE, alpha=0.5)
    ax1.plot(x1, r, linewidth=1.5)
    ax1.plot(x1, rRE, linewidth=1.5)
    ax1.xaxis.set_major_locator(months)
    ax1.xaxis.set_major_formatter(monthFmt)
    ax1.set_ylabel('Survived hours')
    ax1.set_xlabel('Outage start timestep')
    ax1.legend(['Diesel Only', 'Diesel + RE'])
    ax1.set_title(project + "Resilience by timestep")

    plt.ylim((0,float(m+1)/float(n_steps_per_hour)))
    datemin = np.datetime64(x1[0], 'm')
    datemax = np.datetime64(x1[-1], 'm') + np.timedelta64(1, 'm')
    ax1.set_xlim(datemin, datemax)

    plt.savefig(project + "Resiliency{}.png".format(tag))

    fig, ax2 = plt.subplots(figsize=(2200/dpi, 900/dpi), dpi=dpi)
    ax2.plot(x2, y_vals, linewidth=1.5)
    ax2.plot(x2, y_valsRE, linewidth=1.5)
    ax2.set_ylabel("Probability of Surviving Outage")
    ax2.set_xlabel("Duration of Outage (timesteps)")
    ax2.legend(['Diesel Only', 'Diesel+RE'])
    ax2.set_title(project + "Survivability")
    plt.ylim((0,1.1))
    plt.xlim((0,float(m+1)/float(n_steps_per_hour)))
    plt.savefig(project + "Probability of Survival{}.png".format(tag))
    

## User Inputs
<a id='user_input'></a>

Give your simulation a descriptive *project name* and *tag*. 
Project name will show in plot titles, and tag will append at the end of the output file names.

In [ ]:
project = "Sierra Army Depot"
#tag = "200 kW critical load" #commented out here for AUC, moved below

Specify the path for ouput files. None if not specified and the file will be stored in the current directory.

In [ ]:
output_path = None#"//nrel.gov/shared/7A40/Renewable Energy Optimization/REO 3.0/Projects/AUC/Outage simulator/outputs/"

Do you have all your system sizes in a json file? Where is the file?

In [ ]:
read_from_json = True  # True or False
json_input_path = "./SIAD_results_BAU.json"

A valid file may have these fields:
```
    {
        "battCap": 0,
        "battPower": 0,
        "genPower": 1000,
        "fuelAvail": 4000,
        "pvSize": 200,
        "windSize": 0
    }
```

If not, fill in the required system sizes (fill in *0* if the systme is not available/not exist):

In [ ]:
# battCap = 0  # kwh
# #battDuration_list = [0,4,8,12,16,24] #hrs; added for AUC 
# battPower = 0  # kw
# #battPower_list = [400]  # kw
# genPower = 2610  # kw
# fuelAvail = 36000  # gallon
# pvSize = 3100  # kw-DC
# windSize = 0  # kw

#### File locations - critical load, PV production factor, and storage SOC. 
fill in *None* if the data is not available because the system size is 0.

In [ ]:
# load_path = "SAID_load_just_values.csv"  # kw
# #soc_path = "//nrel.gov/shared/7A40/Renewable Energy Optimization/REO 3.0/Projects/AUC/Outage simulator/AUC_SOC-always charged.csv"  # 0 ~ 1.0
# pv_pf_path = "//nrel.gov/shared/7A40/Renewable Energy Optimization/REO 3.0/Projects/AUC/Outage simulator/AUC_PVprodfactor.csv" # 0 ~ 1.0
# wind_pf_path = None # 0 ~ 1.0

## Run simulation and get results
<a id='result'></a>

In [ ]:
tag = str(battPower) + " kW critical load - " + str(dur) + "-hr battery"
main(project, tag, json_input_path, battCap, battPower, genPower, fuelAvail, pvSize, windSize, output_path)
# ownership_model_list = ["third_party","dir_purch"]
# for battPower in battPower_list:
#     #load_path = "//nrel.gov/shared/7A40/Renewable Energy Optimization/REO 3.0/Projects/AUC/Outage simulator/AUC-critload-peak " + str(battPower) +"kw.csv"  # kw
#     for dur in battDuration_list:
#         for ownership_model in ownership_model_list:
#             battCap_tot = dur*battPower  # kwh
#             battCap = (1-0.2)*battCap_tot #usable kwh (considering min SOC)
#             soc_path = "C://Users/kkrah/Documents/GitHub/REopt-API-Analysis/single_site/BreakingBarriers/outputs/batt_soc_"+ ownership_model + str(dur) + "hr-BESS_2019-RTP.csv"
#             tag = str(dur) + "-hr battery-" + ownership_model
#             print(tag, ": ", battCap_tot, battPower, dur, pvSize)
#             main(project, tag, json_input_path, battCap, battPower, genPower, fuelAvail, pvSize, windSize, output_path)

0
1
2
3
4
5
6
7
8
